In [1]:
!pip install crewai==0.32.2 crewai_tools==0.3.0 langchain langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 25.4

In [2]:
# openai_api_key=api_key

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os

openai_api_key = ""
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	# verbose=True
)

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    # verbose=True
)

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    # verbose=True
)

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})


 [2024-06-26 17:30:57][DEBUG]: == Working Agent: Content Planner
 [2024-06-26 17:30:57][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 [2024-06-26 17:31:06][DEBUG]: == [Content Planner] Task output: **Content Plan: Artificial Intelligence Blog Article**

**Outline:**
1. Introduction
- Definition of Artificial Intelligence (AI)
- Importance of AI in today's world
2. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare and finance
3. Key Players in the AI Industry
- Google
- Amazon
- IBM
- Microsoft
4. Noteworthy News in Artificial Intelligence
- Latest AI research breakthroughs
- Impact of AI on job market
5

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: A Look into the Future of Technology

## Introduction

Artificial Intelligence (AI) has become an integral part of our daily lives, revolutionizing the way we interact with technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in various industries, from healthcare to finance. In today's fast-paced world, the importance of AI cannot be overstated, as it enhances efficiency, productivity, and decision-making processes. As we delve deeper into the realm of AI, it is essential to explore the latest trends, key players, and noteworthy news shaping the future of this cutting-edge technology.

## Latest Trends in Artificial Intelligence

The field of AI is constantly evolving, with advancements in machine learning and natural language processing paving the way for innovative applications. Machine learning algorithms are becoming increasingly sophisticated, enabling machines to learn from data and make predictions without explicit programming. Natural language processing, on the other hand, focuses on enabling machines to understand and generate human language, leading to significant developments in virtual assistants and chatbots. In industries such as healthcare and finance, AI is being used to analyze vast amounts of data, improve diagnostics, and enhance customer experiences.

## Key Players in the AI Industry

Several tech giants have emerged as key players in the AI industry, driving innovation and shaping the future of technology. Companies like Google, Amazon, IBM, and Microsoft are at the forefront of AI research and development, investing heavily in AI-powered solutions and services. Google's DeepMind project, Amazon's Alexa virtual assistant, IBM's Watson platform, and Microsoft's Azure AI services are just a few examples of how these companies are leveraging AI to create transformative products and services.

## Noteworthy News in Artificial Intelligence

The AI industry is abuzz with exciting developments, from the latest research breakthroughs to the impact of AI on the job market. Researchers are constantly pushing the boundaries of AI capabilities, exploring new algorithms, and applications that could revolutionize various industries. However, the rise of AI also raises concerns about job displacement and the need for upskilling the workforce to adapt to the changing landscape. It is crucial to stay informed about the latest news and trends in AI to navigate the evolving technology landscape effectively.

In conclusion, Artificial Intelligence is a rapidly evolving field with vast potential to transform our world. By staying informed about the latest trends, key players, and noteworthy news in the AI industry, we can better understand the impact of this revolutionary technology and make informed decisions about its applications. As AI continues to shape the future of technology, it is essential to embrace its potential and harness its power for the benefit of society.

# ***#with hugging_face***

In [35]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="instruction-pretrain/InstructLM-500M",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 500,
        "top_k": 50,
        "temperature": 0.1,
    },
)

tokenizer_config.json:   0%|          | 0.00/940 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [36]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	# verbose=True
    llm=llm
)

In [37]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    # verbose=True
    llm=llm
)

In [38]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    # verbose=True
    llm=llm
)

In [39]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm=llm
)

In [40]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm=llm
)

In [41]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [42]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [43]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
from IPython.display import Markdown
Markdown(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:41:42][DEBUG]: == Working Agent: Content Planner
 [2024-06-26 17:41:42][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 [2024-06-26 17:43:36][DEBUG]: == [Content Planner] Task output: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keyw

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:45:45][DEBUG]: == [Content Writer] Task output: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.


This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
my best complete final answer to the task.
Your final answer must be

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 [2024-06-26 17:47:58][DEBUG]: == [Editor] Task output: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.

This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are pr

my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.

This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.


This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.

This is the expect criteria for your final answer: A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources. 
 you MUST return the actual complete content as the final answer, not a summary.

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought:

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought:

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought:

Thought: